In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats

from sklearn.model_selection import cross_validate, cross_val_score, cross_val_predict, learning_curve,\
train_test_split, GridSearchCV, RandomizedSearchCV, train_test_split
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, OneHotEncoder, FunctionTransformer, LabelEncoder, OrdinalEncoder
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.inspection import permutation_importance
from sklearn.metrics import plot_confusion_matrix, classification_report, precision_recall_curve
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDRegressor, SGDClassifier, Ridge, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.svm import SVC
from sklearn.compose import make_column_selector

## pipeline stuff

from sklearn.pipeline import Pipeline, make_pipeline, make_union
from sklearn.compose import ColumnTransformer, make_column_transformer, make_column_selector
from sklearn import set_config; set_config(display='diagram')



import warnings 
warnings.filterwarnings('ignore')

In [ ]:
from tqdm.auto import tqdm
import itertools

In [3]:
from sklearn.metrics import make_scorer
from sklearn.model_selection import train_test_split

In [ ]:
cat_vars = ['B_30', 
            'B_38', 
            'D_114', 
            'D_116', 
            'D_117', 
            'D_120', 
            'D_126', 
            'D_63', 
            'D_64', 
            'D_66', 
            'D_68']

def_df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_6k_ids_compress.parquet")
# def_df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/defaulter_data_20k_ids_compress.parquet")
pay_df = pd.read_parquet("/Users/sjoerddewit/Desktop/Programming/6 Le Wagon Data Science/final_project/payer_data_20k_ids.parquet")

target_pay = df_pay.groupby('customer_ID').mean()
target_def = df_def.groupby('customer_ID').mean()

target_pay['target'] = int(0)
target_def['target'] = int(1) 

target = pd.concat([target_pay, target_def])


